# Beta Extraction

Replicating LSA regression using scikit-learn just to make sure they're okay.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

if sys.platform == 'darwin':
    cfg = os.path.join("/Users", "njchiang", "CloudStation", "Grad",
                       "Research", "montilab-ucla", "analogy", "config", "project.json")
    plat = "osx"

elif sys.platform == "linux":
    import platform
    if platform.linux_distribution()[0] == "debian":
        cfg = os.path.join("/home", "njchiang", "data", "CloudStation", "Grad",
                           "Research", "montilab-ucla", "analogy", "config", "project.json")
        plat = "linux"
    else:
        cfg = os.path.join("/u", "project", "monti", "Analysis", "Analogy",
                           "code", "analogy", "config", "project.json")
        plat = "hoff"
else:
    cfg = os.path.join("D:\\", "CloudStation", "Grad",
                       "Research", "montilab-ucla", "analogy", "config", "project.json")
    plat = "win"

with open(cfg, "r") as f:
    projectSettings = json.load(f)

paths = projectSettings["filepaths"]["{}Paths".format(plat)]
sys.path.append(paths["github"])
sys.path.append(paths["code"])

In [3]:
from fmri.beta_extract import create_lss_from_lsa, load_aggregated_data, calc_scores

Loading JSON config from D:\CloudStation\Grad\Research\montilab-ucla\analogy\config\analyses.json
Loading JSON config from D:\CloudStation\Grad\Research\montilab-ucla\analogy\config\contrasts.json
Loading label file from: D:\CloudStation\Grad\Research\montilab-ucla\analogy\labels\trialorder_rsa_absorted.csv


In [4]:
from fmri.analogy_utils import analysisSettings, contrastSettings, order, \
    pu, pa, pv, compile_models, rsa, save_rois, load_rois


In [5]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm 
from sklearn.externals.joblib import Parallel, delayed

In [6]:
x = np.random.rand(100, 10)
y = np.random.rand(100, 2)

In [7]:
model = sm.OLS(y[:, 0], x)
results = model.fit()

In [8]:
scores = Parallel(n_jobs=-2)(delayed(calc_scores)(y[:, v], x) for v in range(y.shape[1]))

In [9]:
# Run LSS extraction!
maskname = "grayMatter-bin_mask"
for sub in projectSettings["subjects"]:
    mask = pu.load_img(
        os.path.join(paths["root"], "derivatives", sub, "masks",
                     "{}.nii.gz".format(maskname)), logger=None)
    fmri_data, design = load_aggregated_data(paths, sub, maskname)
    betas = np.zeros([144, fmri_data.shape[1]])
    tstats = np.zeros([144, fmri_data.shape[1]])
    for i, wordpair in enumerate(order.ABTag[::2]):
        print(wordpair)
        regressors = create_lss_from_lsa(design, wordpair, tt="AB")
#         scores = Parallel(n_jobs=-2)(delayed(calc_scores)(fmri_data[:, v], regressors) for v in range(fmri_data.shape[1]))
#         scores = np.array(scores)
#         betas[i] = scores[:, 0]
#         tstats[i] = scores[:, 1]
        betas[i] = LinearRegression().fit(regressors, fmri_data).coef_[:, 0]
    pu.unmask_img(betas, mask).to_filename(
        os.path.join(paths["root"], "derivatives", sub, "betas",
                    "{}_task-analogy_betas-cope-LSS-condensed.nii.gz".format(sub)))
#     pu.unmask_img(tstats, mask).to_filename(
#         os.path.join(paths["root"], "derivatives", sub, "betas",
#                     "{}_task-analogy_betas-tstat-LSS-condensed.nii.gz".format(sub)))

Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\grayMatter-bin_mask.nii.gz
Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\grayMatter-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-01_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\mc\run-01\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-10-17 07:42:47.348655
Ended at 2018-10-17 07:42:47.507657
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-02_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\mc\run-02\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-10-17 07:42:58.912630
Ended at 2018-10-17 07:42:59.028615
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-03_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\mc\run-03\prefiltered_func_data_mcf_fin

start:finish
top:bottom
under:over
up:down
bright:dull
day:evening
enthusiastic:lazy
fun:boring
funny:serious
good:wrong
high:down
just:unfair
loud:discreet
low:up
majority:small
obey:protest
powerful:meek
right:bad
smiling:sad
witty:dumb
accident:damage
bath:cleanliness
disease:sickness
exercise:fitness
explosion:damage
fire:burns
germs:sickness
heater:warmth
illness:discomfort
injury:pain
joke:laughter
loss:grief
repetition:boredom
stimulus:response
tragedy:tears
workout:sweat
anger:yell
coldness:shiver
danger:flee
dirtiness:bathe
dirty:bathe
fright:scream
happiness:smile
heat:sweat
hunger:eat
loneliness:socialize
nervousness:sweat
sadness:cry
sickness:medicate
thirst:drink
thirsty:drink
tiredness:rest
advertise:promote
bathe:clean
breathe:live
burnish:shine
cook:eat
drink:hydrate
exercise:healthy
flee:escape
ignite:burn
read:learn
sleep:rest
speak:express
study:learn
trim:shorten
wash:clean
work:earn
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-03\masks\gray

car:auto
careful:cautious
couch:sofa
cute:adorable
house:home
kid:child
make:manufacture
option:choice
pants:trousers
pretty:beautiful
raise:elevate
run:sprint
spin:twirl
teach:instruct
book:magazine
chair:sofa
fence:hedge
hill:mountain
house:tent
ladder:stairs
paper:parchment
pencil:pen
picture:drawing
pillow:cushion
rake:fork
shovel:spoon
stairs:ladder
sword:knife
table:desk
wagon:trailer
acceleration:speed
darken:color
death:population
dim:light
discount:price
flood:water
force:pressure
heat:temperature
inflation:price
lower:volume
raise:salary
rise:tide
shorten:distance
soften:voice
speed:movement
terror:fear
accept:reject
big:small
black:white
bright:dark
dark:light
difficult:easy
dirty:clean
fast:slow
fat:thin
good:bad
hot:cold
old:young
pretty:ugly
rich:poor
tall:short
warm:cool
ahead:behind
below:above
climb:descend
east:west
forward:backward
front:back
high:low
in:out


MemoryError: 

## Scratch

In [17]:
maskname = "grayMatter-bin_mask"
mask = pu.load_img(
        os.path.join(paths["root"], "derivatives", "sub-01", "masks",
                     "{}.nii.gz".format(maskname)), logger=None)

Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\grayMatter-bin_mask.nii.gz


In [18]:
pu.unmask_img(betas, mask).to_filename(
    os.path.join(paths["root"], "derivatives", "sub-01", "betas",
                "sub-01_task-analogy_betas-LSS-condensed.nii.gz"))

unmasking image...


-4187.989793602079

In [34]:
# sort by columns and take every 2...3?
for tag in test:
    if ":" in tag:
        trialtype = tag.split("_")[1]
        if trialtype == "AB":
            this_tag = tag.split("_")[0].split("::")[0]
        elif trialtype == "CD":
            this_tag = tag.split("_")[0].split("::")[1]
        else: 
            this_tag = tag.split("_")[0]

In [66]:


n_trials = 108
test = des.matrix
i = []
nuisance = test[:, n_trials:]
trial_regs = test[:, :n_trials]
if sum(i) > 1:
    trial = trial_regs[:, i]
    trial_nuisance = np.sum(np.delete(trial_regs, i, axis=1), axis=1, keepdims=True)
else:
    trial = np.zeros([trial_regs.shape[0], 1])
    trial_nuisance = np.sum(trial_regs, axis=1, keepdims=True)

this_design = np.hstack([trial, trial_nuisance, nuisance])

In [70]:
this_design

array([[ 0.00000000e+00,  0.00000000e+00, -3.23345000e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -3.69203000e-03, ...,
         6.83168370e-05,  1.44599190e-03,  3.02412100e-04],
       [ 0.00000000e+00,  0.00000000e+00, -3.35732000e-03, ...,
         3.24496990e-04,  3.25187340e-03,  6.13651980e-04],
       ...,
       [ 0.00000000e+00,  6.43741853e-01, -3.54389000e-04, ...,
         6.70498950e-05,  1.22224080e-04,  4.11822080e-04],
       [ 0.00000000e+00,  6.77502842e-01,  2.78724000e-04, ...,
         4.34175570e-05,  2.71889240e-03,  1.25232470e-03],
       [ 0.00000000e+00,  6.85408745e-01, -3.03946000e-04, ...,
         1.79547110e-05,  2.77381290e-03,  2.04746300e-03]])

In [53]:
labels[labels.ABTag == "Hello Bloop"].index
test[:, i:i+1]

TypeError: slice indices must be integers or None or have an __index__ method

In [8]:
frametimes = np.arange(0, fmri_data.shape[0], tr)
cond_ids = [i for i, r in test_l.iterrows()]

onsets = labels["Onset"]
durations = labels["Duration"]
design_kwargs = {"drift_model": "blank", "add_regs": motion.values}

LSA_des = pa.make_designmat(frametimes, cond_ids, onsets, durations,
                            design_kwargs=design_kwargs, logger=logger)

(864, 21)

In [5]:
for sub, runs in projectSettings["subjects"].items():
    [regress_betas(paths, sub, r, maskname="template-bin_mask", write=True) for r in runs]

Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-01_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\mc\run-01\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 19:57:41.372250
Ended at 2018-09-24 19:57:41.495236
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-02_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\mc\run-02\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 19:58:23.247053
Ended at 2018-09-24 19:58:23.372039
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analog

Creating design matrix at 2018-09-24 20:11:31.008639
Ended at 2018-09-24 20:11:31.126645
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-03\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-03\func\sub-03_task-analogy_run-05_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-03\mc\run-05\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 20:12:11.609079
Ended at 2018-09-24 20:12:11.734064
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-03\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-03\func\sub-03_task-analogy_run-06_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-03\mc\run-06\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 20:12:53.801709
Ended at 2018-09-24 20:12:53.921711
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-03\masks\

Creating design matrix at 2018-09-24 20:26:34.442556
Ended at 2018-09-24 20:26:34.576557
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-06\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-06\func\sub-06_task-analogy_run-01_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-06\mc\run-01\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 20:27:24.578637
Ended at 2018-09-24 20:27:24.716638
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-06\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-06\func\sub-06_task-analogy_run-02_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-06\mc\run-02\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 20:28:11.036736
Ended at 2018-09-24 20:28:11.160734
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-06\masks\

Creating design matrix at 2018-09-24 20:41:19.109938
Ended at 2018-09-24 20:41:19.239938
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-08\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-08\func\sub-08_task-analogy_run-05_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-08\mc\run-05\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 20:42:00.482215
Ended at 2018-09-24 20:42:00.604214
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-08\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-08\func\sub-08_task-analogy_run-06_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-08\mc\run-06\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 20:42:39.712922
Ended at 2018-09-24 20:42:39.831907
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-08\masks\

Creating design matrix at 2018-09-24 20:56:25.183104
Ended at 2018-09-24 20:56:25.320105
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-11\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-11\func\sub-11_task-analogy_run-01_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-11\mc\run-01\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 20:57:14.224903
Ended at 2018-09-24 20:57:14.365904
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-11\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-11\func\sub-11_task-analogy_run-02_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-11\mc\run-02\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 20:58:03.007135
Ended at 2018-09-24 20:58:03.144135
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-11\masks\

Applying mask
Creating design matrix at 2018-09-24 21:12:21.108606
Ended at 2018-09-24 21:12:21.230586
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-13\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-05_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\mc\run-05\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 21:13:08.451634
Ended at 2018-09-24 21:13:08.588637
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-13\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-06_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\mc\run-06\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 21:13:55.380702
Ended at 2018-09-24 21:13:55.511692
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives

Creating design matrix at 2018-09-24 21:27:38.724488
Ended at 2018-09-24 21:27:38.858489
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-16\func\sub-16_task-analogy_run-01_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-16\mc\run-01\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 21:28:21.040622
Ended at 2018-09-24 21:28:21.169622
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\template-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-16\func\sub-16_task-analogy_run-02_events.csv
Loading label file from: D:\fmri\Analogy\derivatives\sub-16\mc\run-02\prefiltered_func_data_mcf_final.par
Applying mask
Creating design matrix at 2018-09-24 21:29:03.582370
Ended at 2018-09-24 21:29:03.705370
unmasking image...
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\

In [5]:
masks_dict = {"ba10": "dfc-left-BA10-bin_mask",
              "lofc": "aal-LOFC-bin_mask",
              "ba46": "dfc-left-BA46-bin_mask",
              "lpitg": "anat-left-pITG-bin_mask",
              "laitg": "anat-left-aITG-bin_mask",
              "lastg": "anat-left-aSTG-bin_mask",
              "lifg": "anat-ba44-45_mask",
              "lips": "anat-LIPS_mask",
              "v1": "aal-V1-bin_mask",
              "csf": "csf-bin_mask"}

In [7]:
save_rois(masks_dict, t="LSA")

Writing sub-01 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-06_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-07_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-08_events.csv
Concatenating 8 images
Read

Reading file from: D:\fmri\Analogy\derivatives\sub-04\masks\csf-bin_mask.nii.gz
Applying mask
Writing sub-05 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-06_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-07_events.csv
centering image
Loading label file from: D:\fmri\Ana

Reading file from: D:\fmri\Analogy\derivatives\sub-08\masks\aal-V1-bin_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-08\masks\csf-bin_mask.nii.gz
Applying mask
Writing sub-09 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-06_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-0

Reading file from: D:\fmri\Analogy\derivatives\sub-12\masks\anat-LIPS_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-12\masks\aal-V1-bin_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-12\masks\csf-bin_mask.nii.gz
Applying mask
Writing sub-13 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-anal

Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\anat-ba44-45_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\anat-LIPS_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\aal-V1-bin_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\csf-bin_mask.nii.gz
Applying mask


In [7]:
save_rois(masks_dict, t="cope-LSS")

Writing sub-01 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-06_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-07_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-08_events.csv
Concatenating 8 images
Read

Reading file from: D:\fmri\Analogy\derivatives\sub-04\masks\csf-bin_mask.nii.gz
Applying mask
Writing sub-05 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-06_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-07_events.csv
centering image
Loading label file from: D:\fmri\Ana

Reading file from: D:\fmri\Analogy\derivatives\sub-08\masks\aal-V1-bin_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-08\masks\csf-bin_mask.nii.gz
Applying mask
Writing sub-09 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-06_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-0

Reading file from: D:\fmri\Analogy\derivatives\sub-12\masks\anat-LIPS_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-12\masks\aal-V1-bin_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-12\masks\csf-bin_mask.nii.gz
Applying mask
Writing sub-13 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-anal

Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\anat-ba44-45_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\anat-LIPS_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\aal-V1-bin_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\csf-bin_mask.nii.gz
Applying mask


In [6]:
save_rois(masks_dict, t="pymvpa")


Writing sub-01 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-06_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-07_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\func\sub-01_task-analogy_run-08_events.csv
Concatenating 8 images
Read

Reading file from: D:\fmri\Analogy\derivatives\sub-04\masks\csf-bin_mask.nii.gz
Applying mask
Writing sub-05 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-06_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-05\func\sub-05_task-analogy_run-07_events.csv
centering image
Loading label file from: D:\fmri\Ana

Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-08\masks\aal-V1-bin_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-08\masks\csf-bin_mask.nii.gz
Applying mask
Writing sub-09 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-09\func\sub-09_task-analogy_run-06_events.csv
centering image
Loading label file from: D:\fmri\Analogy\der

Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-12\masks\anat-LIPS_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-12\masks\aal-V1-bin_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-12\masks\csf-bin_mask.nii.gz
Applying mask
Writing sub-13 betas
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-01_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-02_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-03_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-04_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\sub-13_task-analogy_run-05_events.csv
centering image
Loading label file from: D:\fmri\Analogy\derivatives\sub-13\func\su

Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\anat-ba44-45_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\anat-LIPS_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\aal-V1-bin_mask.nii.gz
Applying mask
Reading file from: D:\fmri\Analogy\derivatives\sub-16\masks\csf-bin_mask.nii.gz
Applying mask
